## Уменьшение размерности и Embedding (manifold)
* Today we are going to learn how to visualize and explore the data
![pictcha](http://sarahannelawless.com/wp-content/uploads/2015/03/tw-1-600x449.jpg)

In [ ]:
from time import time
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

### Загружаем MNIST
 * Если кто не знает, это выборка рукописных цифер

In [ ]:
from sklearn.datasets import load_digits
digits = load_digits(n_class=6)
X = digits.data
y = digits.target
n_samples, n_features = X.shape


In [ ]:
print(X.shape)
print(y.shape)

In [ ]:
# a few testimonials
plt.subplot(1,2,1)
plt.imshow(X[0].reshape(8,8))
plt.subplot(1,2,2)
plt.imshow(X[1].reshape(8,8))

### Визуализация данных

Для начала возьмём случайную пару проекций

In [ ]:
from matplotlib import offsetbox
def plot_embedding(X,y=None,ax=None,show_images=True,min_dist=5e-3,figsize=[12,10]):
    
    #нормализуем данные
    x_min, x_max = np.min(X, 0), np.max(X, 0)
    X = (X - x_min) / (x_max - x_min)

    if ax is None:
        plt.figure(figsize=figsize)
        ax = plt.subplot(1,1,1)

    # рисуем scatter
    if y is None:
        plt.scatter(*X.T)
    else:
        assert y is not None
        #рисуем циферки a-la scatter
        for i in range(X.shape[0]):
            ax.text(X[i, 0], X[i, 1], str(y[i]),
                     color= plt.cm.Set1(y[i] / 10.),
                     fontdict={'weight': 'bold', 'size': 9})

    if not show_images:
        return
        
    shown_images = np.array([[1., 1.]])  # just something big
    for i in range(X.shape[0]):
        dist = np.sum((X[i] - shown_images) ** 2, 1)
        if np.min(dist) < min_dist: continue
        shown_images = np.r_[shown_images, [X[i]]]
        imagebox = offsetbox.AnnotationBbox(
            offsetbox.OffsetImage(digits.images[i], cmap=plt.cm.gray_r),
            X[i])
        ax.add_artist(imagebox)
    plt.xticks([]), plt.yticks([])


### GaussianRandomProjection
 * Выбирает несколько (2) случайных осей в многомерном пространстве данных
 * проецирует выборку на эти оси
 * Сам по себе довольно бесполезен

In [ ]:
from sklearn.random_projection import GaussianRandomProjection

Xrp = GaussianRandomProjection(n_components=2).fit_transform(X)

In [ ]:
plot_embedding(Xrp)

### Wut?!
Нищева нипанятна нащайника!

In [ ]:
plot_embedding(Xrp,y)

После перезапуска Xrp=... результат может поменяться

### Singular Value Decomposition

* Идея: мы пытаемся отобразить данные на такие оси, с которых их потом можно восстановить с минимальными потерями
* Количество этих "новых осей" обычно меньше, чем у изначальных данных

In [ ]:
from sklearn.decomposition import TruncatedSVD

In [ ]:
svd = TruncatedSVD(n_components=2)
Xsvd = svd.fit_transform(X)

In [ ]:
plot_embedding(Xsvd[:,:2],y)

In [ ]:
n = svd.n_components
plt.figure(figsize=[16,8])
for i in range(2):
    plt.subplot(1,2,i+1)
    plt.imshow(svd.components_[i].reshape(8,8),
              cmap='gray',interpolation='none')
    plt.colorbar()


### PCA ака анализ главных компонент
* Идея: мы пытаемся найти оси, вдоль которых расположены данные
* Опять же, количество таких осей меньше, чем в оригинальных данных

__По коду крайне похоже на SVD__ , засим предлагаю применить его самостоятельно

In [ ]:
from sklearn.decomposition import PCA
pca = создай меня с 2 компонентами

Xpca = преобразуй данные

In [ ]:
plot_embedding(Xpca,y)

#### В этот момент хорошей идеей будет сравнить 3 полученные картинки

In [ ]:
plt.figure(figsize=[12,4])
plot_embedding(Xrp,y,ax=plt.subplot(1,3,1),min_dist=3e-2)
plot_embedding(Xsvd,y,ax=plt.subplot(1,3,2),min_dist=3e-2)
plot_embedding(Xpca,y,ax=plt.subplot(1,3,3),min_dist=3e-2)

# LDA aka Linear Discriminant Analysis

* Идея: давайте найдём такие оси, на которых классы (циферки) будут лучше всего разделены
* На самом деле, это самый обычный классификатор, уменьшение размерности - просто побочный эффект

* Интерфейс такой же, как предыдущие, но поскольку этот метод разделяет __классы__, ему нужно отдать, собственно, классы(__y__), а не только признаки(__X__).

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = создай (с 2 компонентами)
Xlda = подгони-преобразуй

In [ ]:
plot_embedding(Xlda,y)

#### Кажется, мы победили!

##### Ан нет, Hard mode enabled

__Сходите, пожалуйста, в начало, и поменяйте у функции load_digits n_class на 10__

После чего, понятное дело, перепрогоните тесты

```

```

```

```

```

```

```

```

```

```

```

```

```

```



# Embedding ака Manifold learning

* В отличие от предыдущих не имеют ограничений вида "только линейное"/"квадратичное"/"синусоидальное" преобразование

* Общая идея: давайте мы попытаемся сопоставить каждой точке в данных новые координаты такие, чтобы "было хорошо"

* Что такое хорошо и как сопоставлять - зависит от конкретного метода

### Multidimensional Scaling

* Идея - давайте расположим новые точки так, чтобы близкие в многомерном пространстве точки получили близки друг к друку координаты, а удалённые - соответственно, далёкие друг от друга

#####  Чуть сложнее
* Давайте зададим точкам такие новые координаты, чтобы попарные расстояния сохранялись как можно точнее.
  * Есть точки в 64-мерном пространстве. Между ними есть расстояние, например
$$ r(a,b) = \sqrt { (a_0 - b_0)^2 + (a_1 - b_1)^2 + ... + (a_63 - b_63)^2}$$
  * Есть наши новые точки в двумерном пространстве, между ними есть расстояние
$$ r_{new}(a',b') = \sqrt { (a'_x - b'_x)^2 + (a'_y - b'_y)^2 } $$

  * Хотим, чтобы $r_{new}$ было максимально близко к $r$
  * Двигаем новые точки так, чтобы в среднем по всем a,b
$$ r_{new}(a',b') - r(a,b) \to min $$

  * Далее по тексту разладка между старым и новым расстоянием называется __stress__

In [ ]:
from sklearn.manifold import MDS
mds = MDS(n_components=2,verbose=2,n_init=1)
Xmds = mds.fit_transform(X)

In [ ]:
plot_embedding(Xmds,y)

# t-SNE
t-distributed Stochasitc Neiborhood Embedding бла-бла-бла

* Идейно похож на MDS, но чуть более пофигист - ему плевать на далёкие точки.

* Хотим, чтобы сохранялось расстояние до K ближайших соседей, остальные - как повезёт

Иными словами, хотим как можно лучше сохранить отношение соседства, забивая на глобальную структуру.

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
%%time
tsne = создай_меня
Xtsne = подгони модель и преобразуй данные

In [ ]:
plot_embedding(Xtsne,y)

# Так кто же лучше?
 * t-SNE зависит от параметра perplexity - на сколько ближайших соседей смотреть (доля)
   * Посмотрите, как результат меняется при изменении perplexity от 1 до 100
   * Все точки брать не обязательно, опорных значений вида [1,5,...50,100] должно хватить 
 


In [ ]:
<посмотри, как меняется картинка tSNE в зависимости от перплексии>

 * Попробуйте сперва выделить несколько главных компонент с PCA, и только потом применить TSNE
   * Для начала, 64D изначально -> 16D после PCA -> 2D после TSNE


In [ ]:
<PCA_или_LDA->tsne>

 * Попробуйте схожие методы - Isomap, LocallyLinearEmbedding или SpectralEmbedding
   * Все лежат в sklearn.manifold

In [ ]:
<Isomap, LLE and Spectral Embedding>